In [23]:
import pandas as pd
import numpy as np
from math import factorial
from scipy import stats
from scipy.optimize import brentq 

In [24]:
df = pd.read_csv("ER Wait Time Dataset.csv",
                 parse_dates=["Visit Date"]) 
df["Hour"] = df["Visit Date"].dt.hour

df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(", "").str.replace(")", "").str.replace("-", "_")

df.head(3)

,visit_id,patient_id,hospital_id,hospital_name,region,visit_date,day_of_week,season,time_of_day,urgency_level,nurse_to_patient_ratio,specialist_availability,facility_size_beds,time_to_registration_min,time_to_triage_min,time_to_medical_professional_min,total_wait_time_min,patient_outcome,patient_satisfaction,hour
0,HOSP-1-20240210-0001,PAT-00001,HOSP-1,Springfield General Hospital,Urban,2024-02-10 20:20:56,Saturday,Winter,Late Morning,Medium,4,3,92,17,22,66,105,Discharged,1,20
1,HOSP-3-20241128-0001,PAT-00002,HOSP-3,Northside Community Hospital,Rural,2024-11-28 02:07:47,Thursday,Fall,Evening,Medium,4,0,38,9,30,30,69,Discharged,3,2
2,HOSP-3-20240930-0002,PAT-00003,HOSP-3,Northside Community Hospital,Rural,2024-09-30 04:02:28,Monday,Fall,Evening,Low,5,1,38,38,40,125,203,Discharged,1,4


In [25]:
print("Bolnice u skupu:", df.hospital_id.unique())

Bolnice u skupu: ['HOSP-1' 'HOSP-3' 'HOSP-2' 'HOSP-5' 'HOSP-4']


In [26]:
# hosp = "HOSP-1"
# df = df[df.hospital_id == hosp].copy()
# print(f"Redova za {hosp}:", len(df))

In [27]:
ia = df["visit_date"].sort_values().diff().dt.total_seconds().dropna() / 60

lmbda_hat = 1 / ia.mean()                     
D, p = stats.kstest(ia, 'expon', args=(0, 1/lmbda_hat))
print(f"KS p = {p:.4f}")
# Ako p > 0.05 -> eksponencijalna raspodjela ⇒ A=M (Poisson dolasci)

KS p = 0.4895


In [28]:
lambda_per_hour = df.groupby("hour").size().mean()
lambda_per_minute = lambda_per_hour / 60
print(f"λ = {lambda_per_minute} (globalno)")

λ = 3.4722222222222223 (globalno)


In [29]:
# mean_service_time = 15   # 15 minuta po pacijentu
# std_service_time = 5
# mu = 1 / mean_service_time  
# print(f"μ = {mu}")
# print(f"σ = {std_service_time}")

In [30]:
W_target = df["total_wait_time_min"].mean()
print(f"Prosječno čekanje W (iz podataka): {W_target:.2f} min (globalno)")

Prosječno čekanje W (iz podataka): 81.92 min (globalno)


In [31]:
c = int(round(df["specialist_availability"].mean()))
print(f"c = {c} (globalno)")

c = 4 (globalno)


In [32]:
df["visit_date"] = pd.to_datetime(df["visit_date"]).dt.date
daily_counts = df.groupby("visit_date").size()
lambda_per_day = daily_counts.mean()
lambda_per_minute = lambda_per_day / (24 * 60)
print(f"λ = {lambda_per_day:.2f} pacijenata/dan ≈ {lambda_per_minute:.4f} pacijenata/min")

λ = 13.70 pacijenata/dan ≈ 0.0095 pacijenata/min


In [33]:
c = int(round(df.groupby("visit_date")["specialist_availability"].mean().mean()))
print(f"c = {c} doktora/dan ≈ {lambda_per_minute:.4f}")

c = 4 doktora/dan ≈ 0.0095


In [34]:
W_target = df.groupby("visit_date")["total_wait_time_min"].mean().mean()
print(f"W = {W_target:.2f} (prosječno vrijeme u sistemu)")

W = 81.34 (prosječno vrijeme u sistemu)


In [35]:
def P_wait(lmbda, mu, c):
    rho = lmbda / (c * mu)
    if rho >= 1:
        return 1
    sum1 = sum((c * rho) ** k / factorial(k) for k in range(c))
    sum2 = (c * rho) ** c / (factorial(c) * (1 - rho))
    return sum2 / (sum1 + sum2)

def Wq_mm(lmbda, mu, c):
    Pw = P_wait(lmbda, mu, c)
    return Pw / (c * mu - lmbda)

def W_total(mu, Cv2=1.0):
    Wq = 0.5 * (1 + Cv2) * Wq_mm(lambda_per_minute, mu, c)
    return Wq + 1 / mu

In [36]:
mu_min = 1 / 120  
mu_max = 1 / 5

try:
    mu_star = brentq(lambda mu: W_total(mu) - W_target, mu_min, mu_max)
    print(f"μ = {mu_star:.4f} (pregleda/min)")
    print(f"Trajanje pregleda (1/μ): {1 / mu_star:.2f} min")
    print(f"Prosječno vrijeme čekanja: {(W_total(mu_star) - (1 / mu_star)):.2f} min/dan")
except ValueError:
    print("Greška: nije moguće naći μ koje daje W ≈", W_target)

μ = 0.0123 (pregleda/min)
Trajanje pregleda (1/μ): 81.12 min
Prosječno vrijeme čekanja: 0.21 min/dan


In [37]:
rho = lambda_per_minute / (c * mu_star)

cv = 1.0

def erlang_c_formula(c, rho):
    sum1 = sum([(c * rho) ** k / factorial(k) for k in range(c)])
    sum2 = (c * rho) ** c / (factorial(c) * (1 - rho))
    P_wait = sum2 / (sum1 + sum2)
    return P_wait

if rho < 1:
    P_wait = erlang_c_formula(c, rho)
    Wq_MMc = P_wait / (c * mu_star - lambda_per_minute)
    Wq = ((cv ** 2) + 1) / 2 * Wq_MMc
    W = Wq + 1 / mu_star
    Lq = lambda_per_minute * Wq
    L = lambda_per_minute * W
else:
    Wq = np.inf
    W = np.inf
    Lq = np.inf
    L = np.inf

In [38]:
print(f"λ = {lambda_per_minute:.4f} dolazaka/min")
print(f"μ = {mu_star:.4f} servisa/min (vrijeme servisa = {1 / mu_star} min)")
print(f"c = {c} doktora")
print(f"ρ = {rho:.4f} iskorištenost")

if rho < 1:
    print(f"Wq = {Wq:.2f} min (prosječno čekanje u redu)")
    print(f"W = {W:.2f} min (ukupno vrijeme u sistemu)")
    print(f"Lq = {Lq:.2f} pacijenata u redu")
    print(f"L = {L:.2f} pacijenata u sistemu")
else:
    print("Sistem je nestabilan jer je ρ ≥ 1 (preopterećen)")

λ = 0.0095 dolazaka/min
μ = 0.0123 servisa/min (vrijeme servisa = 81.12481350379653 min)
c = 4 doktora
ρ = 0.1929 iskorištenost
Wq = 0.21 min (prosječno čekanje u redu)
W = 81.34 min (ukupno vrijeme u sistemu)
Lq = 0.00 pacijenata u redu
L = 0.77 pacijenata u sistemu


In [39]:
df["visit_date"] = pd.to_datetime(df["visit_date"])
lambda_hour = (df.groupby("hour").size() / df["visit_date"].dt.date.nunique()) / 60
print(lambda_hour)

hour
0     0.010411
1     0.009954
2     0.008858
3     0.007808
4     0.008311
5     0.010000
6     0.009726
7     0.009406
8     0.010091
9     0.009817
10    0.010548
11    0.010411
12    0.008995
13    0.009680
14    0.009726
15    0.009087
16    0.010000
17    0.008265
18    0.009361
19    0.009909
20    0.009361
21    0.010365
22    0.009726
23    0.008493
dtype: float64


In [40]:
c_hour = df.groupby("hour")["specialist_availability"].mean().round().astype(int)
print(c_hour)

hour
0     4
1     4
2     4
3     4
4     4
5     4
6     4
7     4
8     4
9     4
10    4
11    4
12    4
13    4
14    4
15    4
16    4
17    4
18    4
19    3
20    4
21    4
22    4
23    4
Name: specialist_availability, dtype: int64
